In [1]:
import requests      # Библиотека работы с HTTP-запросами по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами

In [2]:
# 
import requests 

# Пакет для работы с JSON-файлами
import json

# Модуль для работы со значением времени
import time

# Модуль для работы с операционной системой
import os

In [7]:
def getPages(page = 0):
    '''Функция запроса страницы.
       page - Индекс страницы
       (по умолчанию начинается с первой страницы)'''

       # Справочник для переменной GET-запроса
       # Значения для указания значений в справочнике 
       # указаны в документации к API hh.ru по ссылке: 
       # https://github.com/hhru/api
    job_title = ['data engineer' and 'data analyst' and 'аналитик данных' 
                 and 'аналитик' and 'инженер данных' and 'аналитик']
    params = {
                'text': job_title,
                'area': 1,
                'page': page,
                'per_page': 100
             }

    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode()
    req.close()
    return data

        

# Считаем первые Х вакансий
for page in range(0,100):
    jsObj = json.loads(getPages(page))

    # Сохраняем файлы в папку {путь до текущего документа со скриптом}\установить\ваш\путь
    # Определяем количество файлов в папке для сохранения документа с ответом запроса
    # Полученное значение используем для формирования имени документа
    nextFileName = 'vacancies_parsing/{}.json'.format(len(os.listdir('vacancies_parsing')))

    # Создаем новый документ, записываем в него ответ запроса, после закрываем
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False, indent=3))

    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObj['pages'] - page) <= 1:
        break

    time.sleep(0.25)


print('Вакансии по запросу собраны')

Вакансии по запросу собраны


In [25]:
import json
import os
import time
import requests

def get_vacancies(job_title, page=0):
    '''Функция запроса страницы.
       job_title - наименование вакансии
       page - Индекс страницы
       (по умолчанию начинается с первой страницы)'''

    # Справочник для переменной GET-запроса
    # Значения для указания значений в справочнике 
    # указаны в документации к API hh.ru по ссылке: 
    # https://github.com/hhru/api
    params = {
        'text': job_title,
        'area': 1,
        'page': page,
        'per_page': 100
    }

    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode()
    req.close()
    return data

# Собираем все вакансии в один JSON-файл
vacancies = []
vacancies_ids = set()
job_titles = ['data engineer', 'data analyst', 'data scientist', 'аналитик данных', 'аналитик', 'инженер данных']

for job_title in job_titles:
    for page in range(0, 100):
        response = json.loads(get_vacancies(job_title, page))
        items = response['items']
        for item in items:
            if item['id'] not in vacancies_ids:
                # добавляем вакансию в список
                vacancies.append(item)
                # добавляем id вакансии во множество уже просмотренных id
                vacancies_ids.add(item['id'])
        # Проверка на последнюю страницу, если вакансий меньше 10000
        if response['pages'] - page <= 1:
            break
        time.sleep(0.25)

# Сохраняем полученные вакансии в файле
with open('data/vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(vacancies, f, ensure_ascii=False, indent=4)

print('Вакансии по запросу собраны и сохранены в файл vacancies.json')
# Количество вакансий
print(f"Количество собранных вакансий: {len(vacancies)}")

# Уникальные значения ключа 'name'
names = set([v['name'] for v in vacancies])
print(f"Уникальные названия вакансий: {names}")


Вакансии по запросу собраны и сохранены в файл vacancies.json
Количество собранных вакансий: 6286
Уникальные названия вакансий: {'Аналитик (расходы)', 'Ведущий аналитик данных/Архитектор данных DG DQ (ДИР)', 'Архитектор ИБ по сетевым технологиям (Ведущий инженер-проектировщик)', 'Аналитик в Дирекцию по логистике', 'BI-аналитик (QlikSense)', 'Product Analyst (GameDev)', 'Специалист-аналитик в департамент стратегического развития', 'Senior/ Team Lead Data Scientist в ML-команду', 'DevOps engineer (удаленно)', 'Ведущий разработчик Golang [Media-Push]', 'Инженер службы по эксплуатации слаботочных систем', 'Middle+ Системный аналитик', 'Специалист сметно-договорного отдела (Инженер-сметчик)', 'Инженер-проектировщик КИПиА/АСУТП', 'Системный аналитик Центра HR TECH (Проект HCM Атомкор)', 'Ведущий системный администратор/DevOps engineer', 'Младший финансовый аналитик', 'Доцент Департамента анализа данных и машинного обучения', 'Главный инженер проекта (проектирование)', 'Cost Controller', 'Рук

In [28]:
# Проверим, попала ли в выборку вакансия, опубликованная на hh одной из последних:

# Открываем файл с вакансиями и загружаем их в переменную vacancies
with open('data/vacancies.json', 'r', encoding='utf-8') as f:
    vacancies = json.load(f)

# ID вакансии, которую нужно найти
vacancy_id = '79150358'

# Поиск вакансии по ID и вывод ее описания
for vacancy in vacancies:
    if vacancy['id'] == vacancy_id:
        print("Обязанности:\n", vacancy['snippet']['responsibility'])
        print("\nТребования:\n", vacancy['snippet']['requirement'])
        break
else:
    print(f"Вакансия с номером {vacancy_id} не найдена")


Обязанности:
 Разработка аналитических витрин и визуализация дашбордов в BI решении по направлению <highlighttext>аналитика</highlighttext> розничного бизнеса в сети офисов продаж. 

Требования:
 Умение и желание работать с данными. Аналитический склад ума. Развитое логическое мышление. Опыт <highlighttext>аналитики</highlighttext> и сбора статистики по розничным продуктам...


In [ ]:
number_of_pages = 1000
#number_of_ads = number_of_pages * per_page

job_title = ["'Data Analyst' and 'data scientist'"]
for job in job_title:
	data=[]
	for i in range(number_of_pages):
		url = 'https://api.hh.ru/vacancies'
		par = {'text': job, 'area':'113','per_page':'10', 'page':i}
		r = requests.get(url, params=par)
		e=r.json()
		data.append(e)
        vacancy_details = data[0]['items'][0].keys()
		df = pd.DataFrame(columns= list(vacancy_details))
		ind = 0
		for i in range(len(data)):
			for j in range(len(data[i]['items'])):
				df.loc[ind] = data[i]['items'][j]
				ind+=1
	csv_name = job+".csv"
	df.to_csv(csv_name)

In [ ]:
# импортируем библиотеку request для работы с данными в сети
import requests
x=[]
all_zp = 0
all_n = 0
#цикл, который скачивает вакансии
for i in range(200):
    # запрос
    url = 'https://api.hh.ru/vacancies'
    #параметры, которые будут добавлены к запросу
    par = {'text': 'Data Analyst' and 'Data Scientist' and 'Аналитик данных', 'area':'113','per_page':'10', 'page':i}
    r = requests.get(url, params=par)
    e=r.json()
    x.append(e)
for j in x:
    y = j['items']
    #объявляем переменную n для подсчета, количества итераций цикла перебирающего зарплаты в вакансиях
    n=0
    #объявляем переменную sum_zp для подсчета, суммы зарплат в вакансиях
    sum_zp=0
    #цикл, переберает объекты, т.е перебирает вакансии
    for i in y:
        # проверяем есть ли значения в словаре по ключу salary. Т.е проверяем есть ли в вакансии данные по зарплате
        if i['salary'] !=None:
            #записываем значение в переменную s
            s=i['salary']
            # проверяем есть ли значения по ключу from. Т.е проверяем есть ли в вакансии данные по минимальной зп
            if s['from'] !=None:
                # считаем количество обработанных вакансий в которых указана минимальная ЗП
                n+=1
                #получаем минимальную ЗП по ключу from
                s['from']
                #считаем сумму ЗП по вакансиям
                sum_zp +=s['from']
    #добавляем сумму зп по итерации цикла
    all_zp +=sum_zp
    #добавляем сумму n по итерации цикла
    all_n +=n
    #считаем среднюю ЗП
av_zp=all_zp/all_n
print(av_zp)

In [ ]:
def getPages(page = 0):
    '''Функция запроса страницы.
       page - Индекс страницы
       (по умолчанию начинается с первой страницы)'''

       # Справочник для переменной GET-запроса
       # Значения для указания значений в справочнике 
       # указаны в документации к API hh.ru по ссылке: 
       # https://github.com/hhru/api
    params = {
                'text': 'data engineer',
                'area': 1,
                'page': page,
                'per_page': 100}

    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode()
    req.close()
    return data

In [ ]:
df = pd.read_json('data/0.json')

In [ ]:
df.head(10)

In [ ]:
df['items'].values[4]

In [ ]:
import json
# load data using Python JSON module
with open('data/0.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
df = pd.json_normalize(data, record_path =['items'])
df

In [ ]:
requirement = df.at[4, 'snippet.requirement']

In [ ]:
requirement

In [ ]:
len(requirement)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df

In [ ]:
dt = []
for files in os.listdir('data/'):
    f = open('data/{}'.format(files), encoding='utf8')
    jsonText = f.read()
    f.close()
    jsonObj = json.loads(jsonText)
    if jsonObj['found'] != 0:
        for js in jsonObj['items']:
            if js['salary'] != None:
                salary_from = js['salary']['from']
                salaty_to = js['salary']['to']
            else:
                salary_from = None
                salaty_to = None
            if js['address'] != None:
                address_raw = js['address']['raw']
            else:
                address_raw = None
            if js['schedule'] != None:
                schedule_name = js['schedule']['name']
            else:
                schedule_name = None
            if js['department'] != None:
                department_name = js['department']['name']
            else:
                department_name = None
            dt.append([
                js['id'],
                js['premium'],
                js['name'],
                js['department']['name'],
                js['has_test'],
                js['response_letter_required'],
                js['area']['id'],
                js['area']['name'],
                salary_from, 
                salaty_to,
                js['type']['name'],
                address_raw,
                js['response_url'],
                js['sort_point_distance'],
                js['published_at'],
                js['created_at'],
                js['archived'],
                js['apply_alternate_url'],
                js['insider_interview'],
                js['url'],
                js['alternate_url'],
                js['relations'],
                #js['employer']['id'],
                js['employer']['name'],
                js['snippet']['requirement'],
                js['snippet']['responsibility'],
                js['contacts'],
                #js['schedule']['name'],
                js['working_days'],
                js['working_time_intervals'],
                js['working_time_modes'],
                js['accept_temporary']
                ])

In [ ]:
df = pd.DataFrame(dt, columns = [
                                'id',
                                'premium',
                                'name',
                                #'department_name',
                                'has_test',
                                'response_letter_required',
                                'area_id',
                                'area_name',
                                'salary_from', 
                                'salaty_to',
                                'type_name',
                                'address_raw',
                                'response_url',
                                'sort_point_distance',
                                'published_at',
                                'created_at',
                                'archived',
                                'apply_alternate_url',
                                'insider_interview',
                                'url',
                                'alternate_url',
                                'relations',
                                'employer_id',
                                #'employer_name',
                                'snippet_requirement',
                                'snippet_responsibility',
                                'contacts',
                                #'schedule_name',
                                'working_days',
                                'working_time_intervals',
                                'working_time_modes',
                                'accept_temporary'
                                ])   

In [ ]:
import requests
import json

url = 'https://api.hh.ru/vacancies'
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
#            'Authorization': 'Bearer YOUR_API_KEY'}

params = {
    'text': 'data engineer',
    'area': 1, # Код региона (Москва)
    'per_page': 100 # Количество вакансий на страницу
}

response = requests.get(url, params=params)#headers=headers, params=params)

# Обработка ответа
if response.ok:
    data = json.loads(response.text)
    vacancies = data['items']
    df = pd.DataFrame(vacancies)
    #print(df[['name', 'salary']])
else:
    print(response.status_code, response.text)


In [ ]:
df.info()

In [ ]:
df.at[1, 'snippet']

In [ ]:
df.head(10)

In [ ]:
import pandas as pd
import requests

# Создаем пустой DataFrame
vacanciesdf = pd.DataFrame(columns=['Name', 'Currency', 'From', 'Area', 'Requirement'])

# Параметры запроса
params = {
    'text': 'data engineer',
    'area': 1, # Код региона (Москва)
    'per_page': 100 # Количество вакансий на страницу
}

# Получаем данные со всех страниц
for pageNum in range(21):
    response = requests.get('https://api.hh.ru/vacancies', params=params)
    if response.ok:
        data = response.json()
        if data['items']:
            items = data['items']
            for item in items:
                # Добавляем строку в DataFrame
                vacanciesdf = vacanciesdf.append({
                    'Name': item['employer']['name'],
                    'Currency': item['salary']['currency'] if item['salary'] else None,
                    'From': item['salary']['from'] if item['salary'] else None,
                    'Area': item['area']['name'] if item['area'] else None,
                    'Requirement': item['snippet']['requirement'] if item['snippet'] else None
                }, ignore_index=True)
            params['page'] = pageNum + 1
            print(f"Upload pages: {pageNum + 1}")
        else:
            break
    else:
        print(f"Error {response.status_code}: {response.text}")
    # Задержка в 3 секунды между запросами
    time.sleep(3)


In [ ]:
vacanciesdf.head(10)

In [ ]:
vacanciesdf.at[0, 'Requirement']